In [43]:
path = '/kaggle/input/datasetactor/wallets_features_classes_combined.csv'

In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier

In [45]:
df = pd.read_csv(path)

In [46]:
df_wallets_classification = df
df_wallets_classification = df_wallets_classification.drop(columns=['Time step']).drop_duplicates()
df_wallets_classification

,address,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.0,0.000000,0.0,0.0,24.0,1.0,1.0,1.0,1.0
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
9,1111VHuXEzHaRCgXbVwojtaP7Co3QABb,2,0.0,1.0,431522.0,431522.0,0.0,1.0,0.0,431522.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
10,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,3,1.0,1.0,423456.0,423456.0,0.0,2.0,423456.0,423456.0,...,0.0,0.0,0.000000,0.0,0.0,4.0,1.0,1.0,1.0,1.0
12,1115LWW3xsD9jT9VRY7viCN9S34RVAAuA,2,0.0,1.0,429513.0,429513.0,0.0,1.0,0.0,429513.0,...,0.0,0.0,0.000000,0.0,0.0,5.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268254,3R2Uw5MRdSSigp8AjfT7K5es6Hupm4qLSq,3,1.0,1.0,451684.0,451685.0,1.0,2.0,451685.0,451684.0,...,0.0,0.0,0.000000,0.0,0.0,3.0,1.0,1.0,1.0,1.0
1268256,3R2VBFbqHGC4bQ7b4ixN4jZTdv7RMbEYtf,3,0.0,1.0,477895.0,477895.0,0.0,1.0,0.0,477895.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268257,3R2WFmRwbDeo3rMVVu5J3jjMxAuQYYWAid,3,0.0,1.0,407342.0,407342.0,0.0,1.0,0.0,407342.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268258,3R2WTZGYLmbJQyoDSBftJsPRvF1mSEtkh6,3,0.0,1.0,395235.0,395235.0,0.0,1.0,0.0,395235.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [47]:
data = df_wallets_classification.loc[df_wallets_classification['class'] != 3, 'address']
df_final = df_wallets_classification.loc[df_wallets_classification['address'].isin(data)]

In [48]:
data

0           111112TykSw72ztDN2WJger4cynzWYC5w
9            1111VHuXEzHaRCgXbVwojtaP7Co3QABb
12          1115LWW3xsD9jT9VRY7viCN9S34RVAAuA
13          1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ
15          111ECNFEjJsQJ6LkAsAESBGTHBvZE66oX
                          ...                
1268243    3R2KQxz2Uk7xF9bfDGffTn52NrfnBjdaLy
1268244    3R2LAgPtnjeU2XdyR1GUXaTzKmh81VETYm
1268251    3R2SZEK2UggNm1ntkXTe8eoea11sxg9vjw
1268252    3R2TfX1PDN93SHbDXZqYPCn1cgmCeLMBra
1268253    3R2URFhybtzYckKxbP17qMYN989fWC8quM
Name: address, Length: 265354, dtype: object

In [49]:
df_final

,address,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,1.0,1.0
9,1111VHuXEzHaRCgXbVwojtaP7Co3QABb,2,0.0,1.0,431522.0,431522.0,0.0,1.0,0.0,431522.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
12,1115LWW3xsD9jT9VRY7viCN9S34RVAAuA,2,0.0,1.0,429513.0,429513.0,0.0,1.0,0.0,429513.0,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
13,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,2,1.0,1.0,399269.0,399271.0,2.0,2.0,399271.0,399269.0,...,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0
15,111ECNFEjJsQJ6LkAsAESBGTHBvZE66oX,2,0.0,1.0,423462.0,423462.0,0.0,1.0,0.0,423462.0,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268243,3R2KQxz2Uk7xF9bfDGffTn52NrfnBjdaLy,2,0.0,1.0,411364.0,411364.0,0.0,1.0,0.0,411364.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0
1268244,3R2LAgPtnjeU2XdyR1GUXaTzKmh81VETYm,2,0.0,1.0,435555.0,435555.0,0.0,1.0,0.0,435555.0,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
1268251,3R2SZEK2UggNm1ntkXTe8eoea11sxg9vjw,2,0.0,1.0,421443.0,421443.0,0.0,1.0,0.0,421443.0,...,0.0,0.0,0.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0
1268252,3R2TfX1PDN93SHbDXZqYPCn1cgmCeLMBra,2,0.0,1.0,437572.0,437572.0,0.0,1.0,0.0,437572.0,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0


In [50]:
y = df_final[['class']]
y = y['class'].apply(lambda x: 0 if x == 2 else 1)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size = 0.3, random_state = 15, shuffle=False)
X_train = X_train.drop(columns=['address', 'class'])
X_test = X_test.drop(columns=['address', 'class'])

In [52]:
X_train

,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,1.0,0.010628,...,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,1.0,1.0
9,0.0,1.0,431522.0,431522.0,0.0,1.0,0.0,431522.0,1.0,0.000104,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
12,0.0,1.0,429513.0,429513.0,0.0,1.0,0.0,429513.0,1.0,0.000416,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
13,1.0,1.0,399269.0,399271.0,2.0,2.0,399271.0,399269.0,1.0,0.230800,...,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0
15,0.0,1.0,423462.0,423462.0,0.0,1.0,0.0,423462.0,1.0,0.010000,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878995,0.0,1.0,477895.0,477895.0,0.0,1.0,0.0,477895.0,1.0,0.699900,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
879009,0.0,1.0,461769.0,461769.0,0.0,1.0,0.0,461769.0,1.0,0.001074,...,0.0,0.0,0.0,0.0,0.0,15.0,1.0,1.0,1.0,1.0
879013,1.0,0.0,451683.0,451683.0,0.0,1.0,451683.0,0.0,1.0,0.009718,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0
879014,0.0,1.0,405318.0,405318.0,0.0,1.0,0.0,405318.0,1.0,0.014834,...,0.0,0.0,0.0,0.0,0.0,10.0,1.0,1.0,1.0,1.0


In [53]:
X_test

,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
879023,1.0,1.0,461766.0,461767.0,1.0,2.0,461767.0,461766.0,1.0,0.041183,...,0.0,0.0,0.0,0.0,0.0,7.0,1.0,1.0,1.0,1.0
879025,1.0,1.0,437579.0,437580.0,1.0,2.0,437580.0,437579.0,1.0,0.014200,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0
879044,1.0,0.0,399265.0,399265.0,0.0,1.0,399265.0,0.0,1.0,0.023719,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
879045,1.0,1.0,391211.0,391212.0,1.0,2.0,391212.0,391211.0,1.0,0.021977,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
879059,1.0,0.0,453700.0,453700.0,0.0,1.0,453700.0,0.0,1.0,0.004768,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268243,0.0,1.0,411364.0,411364.0,0.0,1.0,0.0,411364.0,1.0,0.000140,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0
1268244,0.0,1.0,435555.0,435555.0,0.0,1.0,0.0,435555.0,1.0,0.000305,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
1268251,0.0,1.0,421443.0,421443.0,0.0,1.0,0.0,421443.0,1.0,0.004600,...,0.0,0.0,0.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0
1268252,0.0,1.0,437572.0,437572.0,0.0,1.0,0.0,437572.0,1.0,0.031884,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0


In [54]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [55]:
#These columns collected in feature selection and retrain phase
columns_selected_from_rf = ['fees_min',
 'transacted_w_address_total',
 'fees_as_share_max',
 'last_block_appeared_in',
 'blocks_btwn_txs_max',
 'first_received_block',
 'fees_total',
 'fees_median',
 'first_block_appeared_in',
 'first_sent_block',
 'fees_mean',
 'fees_max',
 'btc_sent_max',
 'transacted_w_address_median',
 'fees_as_share_min',
 'total_txs']

In [56]:
X_train_selected_rf = X_train_scaled[columns_selected_from_rf]
X_test_selected_rf = X_test_scaled[columns_selected_from_rf]

In [57]:
columns_selected_from_xgb = ['fees_min',
 'transacted_w_address_total',
 'fees_as_share_max',
 'last_block_appeared_in',
 'blocks_btwn_txs_max',
 'first_received_block',
 'fees_total',
 'fees_median',
 'first_block_appeared_in',
 'first_sent_block',
 'fees_mean',
 'fees_max',
 'btc_sent_max',
 'transacted_w_address_median',
 'fees_as_share_min',
 'total_txs',
 'btc_sent_median',
 'btc_transacted_min']

In [58]:
X_train_selected_xgb = X_train_scaled[columns_selected_from_xgb]
X_test_selected_xgb = X_test_scaled[columns_selected_from_xgb]

# Train model again with feature from rf

In [59]:
def get_model_metrics(y_true, y_pred):
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, zero_division=0)
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    return {
        'Precision': prec[1],
        'Recall': rec[1],
        'F1 Score': f1[1],
        'Micro F1': micro_f1,
        'Accuracy': accuracy
    }

In [60]:
results = []

## LR

In [61]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_selected_rf, y_train)
y_pred_lr = lr.predict(X_test_selected_rf)
metrics_lr = get_model_metrics(y_test, y_pred_lr)
metrics_lr['Model'] = 'Logistic Regression'
results.append(metrics_lr)

## RF

In [62]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_selected_rf, y_train)
y_pred_rf = rf.predict(X_test_selected_rf)
metrics_rf = get_model_metrics(y_test, y_pred_rf)
metrics_rf['Model'] = 'Random Forest'
results.append(metrics_rf)

## XGB

In [63]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_selected_rf, y_train)
y_pred_xgb = xgb.predict(X_test_selected_rf)
metrics_xgb = get_model_metrics(y_test, y_pred_xgb)
metrics_xgb['Model']='XGB'
results.append(metrics_xgb)

## MLP

In [64]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train_selected_rf, y_train)
y_pred_mlp = mlp.predict(X_test_selected_rf)
metrics_mlp = get_model_metrics(y_test, y_pred_mlp)
metrics_mlp['Model'] = 'Multi-Layer Perceptron'
results.append(metrics_mlp)

## LGBM

In [65]:
lgbm = LGBMClassifier(random_state=42, n_estimators=100)
lgbm.fit(X_train_selected_rf, y_train)
y_pred_lgbm = lgbm.predict(X_test_selected_rf)
metrics_lgbm = get_model_metrics(y_test, y_pred_lgbm)
metrics_lgbm['Model']='LGBM'
results.append(metrics_lgbm)

[LightGBM] [Info] Number of positive: 11377, number of negative: 174370
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3496
[LightGBM] [Info] Number of data points in the train set: 185747, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061250 -> initscore=-2.729586
[LightGBM] [Info] Start training from score -2.729586


## Ensemble model

In [66]:
ensembles = {
    "RF + MLP": [("rf", rf), ("mlp", mlp)],
    "RF + XGB": [("rf", rf), ("xgb", xgb)],
    "MLP + XGB": [("mlp", mlp), ("xgb", xgb)],
    "RF + MLP + XGB": [("rf", rf), ("mlp", mlp), ("xgb", xgb)],
}

In [67]:
for name, estimators in ensembles.items():
    clf = VotingClassifier(estimators=estimators, voting='hard')
    clf.fit(X_train_selected_rf, y_train)
    y_pred = clf.predict(X_test_selected_rf)
    metrics_ensemble = get_model_metrics(y_test, y_pred)
    metrics_ensemble['Model']= name
    results.append(metrics_ensemble)

In [68]:
df_results = pd.DataFrame(results)
df_results = df_results[['Model', 'Precision', 'Recall', 'F1 Score', 'Micro F1', 'Accuracy']]  # order columns

print(df_results)

                    Model  Precision    Recall  F1 Score  Micro F1  Accuracy
0     Logistic Regression   0.510972  0.056421  0.101621  0.963797  0.963797
1           Random Forest   0.896992  0.825891  0.859975  0.990240  0.990240
2                     XGB   0.889905  0.808584  0.847298  0.989423  0.989423
3  Multi-Layer Perceptron   0.797255  0.442368  0.569012  0.975681  0.975681
4                    LGBM   0.883013  0.762894  0.818570  0.987727  0.987727
5                RF + MLP   0.956851  0.437522  0.600475  0.978871  0.978871
6                RF + XGB   0.932811  0.783316  0.851552  0.990089  0.990089
7               MLP + XGB   0.961098  0.436137  0.600000  0.978896  0.978896
8          RF + MLP + XGB   0.920243  0.786777  0.848293  0.989787  0.989787


# Train model again with feature from xgb 

## LR

In [69]:
results_a = []

In [70]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_selected_xgb, y_train)
y_pred_lr = lr.predict(X_test_selected_xgb)
metrics_lr = get_model_metrics(y_test, y_pred_lr)
metrics_lr['Model'] = 'Logistic Regression'
results_a.append(metrics_lr)

## RF

In [71]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_selected_xgb, y_train)
y_pred_rf = rf.predict(X_test_selected_xgb)
metrics_rf = get_model_metrics(y_test, y_pred_rf)
metrics_rf['Model'] = 'Random Forest'
results_a.append(metrics_rf)

## XGB

In [72]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_selected_xgb, y_train)
y_pred_xgb = xgb.predict(X_test_selected_xgb)
metrics_xgb = get_model_metrics(y_test, y_pred_xgb)
metrics_xgb['Model']='XGB'
results_a.append(metrics_xgb)

## MLP

In [73]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train_selected_xgb, y_train)
y_pred_mlp = mlp.predict(X_test_selected_xgb)
metrics_mlp = get_model_metrics(y_test, y_pred_mlp)
metrics_mlp['Model'] = 'Multi-Layer Perceptron'
results_a.append(metrics_mlp)

## LGBM

In [74]:
lgbm = LGBMClassifier(random_state=42, n_estimators=100)
lgbm.fit(X_train_selected_xgb, y_train)
y_pred_lgbm = lgbm.predict(X_test_selected_xgb)
metrics_lgbm = get_model_metrics(y_test, y_pred_lgbm)
metrics_lgbm['Model']='LGBM'
results_a.append(metrics_lgbm)

[LightGBM] [Info] Number of positive: 11377, number of negative: 174370
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4006
[LightGBM] [Info] Number of data points in the train set: 185747, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061250 -> initscore=-2.729586
[LightGBM] [Info] Start training from score -2.729586


## Ensemble model

In [75]:
ensembles = {
    "RF + MLP": [("rf", rf), ("mlp", mlp)],
    "RF + XGB": [("rf", rf), ("xgb", xgb)],
    "MLP + XGB": [("mlp", mlp), ("xgb", xgb)],
    "RF + MLP + XGB": [("rf", rf), ("mlp", mlp), ("xgb", xgb)],
}

In [76]:
for name, estimators in ensembles.items():
    clf = VotingClassifier(estimators=estimators, voting='hard')
    clf.fit(X_train_selected_xgb, y_train)
    y_pred = clf.predict(X_test_selected_xgb)
    metrics_ensemble = get_model_metrics(y_test, y_pred)
    metrics_ensemble['Model']= name
    results_a.append(metrics_ensemble)

In [77]:
df_results = pd.DataFrame(results_a)
df_results = df_results[['Model', 'Precision', 'Recall', 'F1 Score', 'Micro F1', 'Accuracy']]  # order columns

print(df_results)

                    Model  Precision    Recall  F1 Score  Micro F1  Accuracy
0     Logistic Regression   0.511041  0.056075  0.101061  0.963797  0.963797
1           Random Forest   0.905602  0.816892  0.858963  0.990265  0.990265
2                     XGB   0.892220  0.813776  0.851195  0.989674  0.989674
3  Multi-Layer Perceptron   0.771987  0.410177  0.535714  0.974198  0.974198
4                    LGBM   0.890581  0.769124  0.825409  0.988192  0.988192
5                RF + MLP   0.938003  0.403254  0.564028  0.977376  0.977376
6                RF + XGB   0.933993  0.783662  0.852249  0.990139  0.990139
7               MLP + XGB   0.951020  0.403254  0.566359  0.977590  0.977590
8          RF + MLP + XGB   0.916165  0.790585  0.848755  0.989775  0.989775
